# GammaBayes Data Classes

One of the more fundamental things within `GammaBayes` is the use of three data classes: `EventData`, `Parameter`, and `ParameterSet`. Here we will go over `Parameter` and `ParameterSet`.

# `Parameter`

The `Parameter` class is a nice wrapper (sensing a theme?) for important values involved when performing analysis on a parameter. This class provides less help in the way of keyword arguments as it is meant to act more like a dictionary of common keys than a class almost. Two example inputs are below.

In [1]:
import numpy as np

example_discrete_parameter_dict = {
    'discrete': True,
    'name': 'mass',
    'parameter_type': 'spectral',
    'scaling': 'log10',
    'bounds': [1e-1, 1e2],
    'default_value': 1.0,
    'bins': 31,
}

example_continuous_parameter_dict = {
    'discrete': False,
    'name': 'sig|total',
    'scaling': 'linear',
    'bounds': [0,1],
}

So when we have two types of parameters continuous and discrete. 

When we analyse discrete parameters we need to have the range of values that the parameter can take. In the case of the mass parameter, based on the information provided, we would construct an axis of values like so.

In [2]:
mass_axis = np.logspace(np.log10(1e-1), np.log10(1e2), 31)

We also need to obviously keep track of some identifier for each parameter, e.g. mass. 

If we may also want to keep track of a default value or standard value for a parameter (e.g. 0.17 for the alpha parameter in the Einasto dark matter density profile).

And if we plug this parameter into some sort of MC sampler (e.g. nested sampling) we would want a nice way to keep track of the inverse cdf to turn a unit cube into a value of said parameter.

To make my, and your, lives each easier, this class is just meant to put all of this into a single semi-standardised object.

In [3]:
from gammabayes import Parameter

mass_parameter = Parameter(example_discrete_parameter_dict)

And we can access all these attributes like a dictionary.

In [4]:
mass_parameter['name']

'mass'

Or you can access them as attributes of the class.

In [5]:
mass_parameter.name

'mass'

And derivatives of these values as attributes/properties

In [6]:
mass_parameter.axis

array([  0.1       ,   0.12589254,   0.15848932,   0.19952623,
         0.25118864,   0.31622777,   0.39810717,   0.50118723,
         0.63095734,   0.79432823,   1.        ,   1.25892541,
         1.58489319,   1.99526231,   2.51188643,   3.16227766,
         3.98107171,   5.01187234,   6.30957344,   7.94328235,
        10.        ,  12.58925412,  15.84893192,  19.95262315,
        25.11886432,  31.6227766 ,  39.81071706,  50.11872336,
        63.09573445,  79.43282347, 100.        ])

And based on the input parameters we create an inverse cumulative distribution function that can be used within a sampler within the 'transform' method. This is pretty much the main reason the class exists.

In [7]:
mass_parameter.transform(0.1)

0.19952623149688797

This also allows one to specify all needed information from a yaml file, as the class just takes in a dictionary, and you can store other information about the parameter as well as it is essentially just a dictionary.

In [8]:
mass_parameter['cool'] = True

In [9]:
mass_parameter.cool

True

We also note that we generally keep track of a parameter called `parameter_type` which currently takes values of `spectral` or `spatial`. This is because most parameters can be put into one of these two categories, and many models treat them independently (e.g. dark matter's angular vs spectral distributions). So to decrease the number of unneeded computations (which is one of the main focuses of this code) one can read in the relevant parameters to each component without any checks as they are already stored in a formatted manner.

## Scaling

The `scaling` parameter can either take the value of `linear` or `log10`. Which for discrete parameters means either using `np.linspace` or `np.logspace` and for continuous parameters different transform functions are used of the kind,

`u * transform_scale + bounds[0]`


or

`10**(u * transform_scale + np.log10(bounds[0]))`


And `transform_scale` is either `np.diff(bounds)` or `np.log10(bounds)` depending on which scale is chosen.

## Prior ID/Likelihood ID

Another parameter that the class keeps track of are identifiers for the prior or likelihood that the parameter belong to. This is typically just a string, but helps keep track of where the parameter belongs within the analysis.

And just a note, likelihood parameter analysis is not inherently supported within the code yet. Currently the focys of the code is to optimise the evaluation of the priors. Later updates may include this functionality.

And if you're interested in doing this yourself or want to help out, please email me at `Liam.Pinchbeck@monash.edu`.

## Save/Load

And just like most classes in this package, you can save and/or load.

In [10]:
mass_parameter.save('mass_param.h5')

In [11]:
loaded_mass_parameter = Parameter.load('mass_param.h5')

{'bins': 31, 'cool': True, 'default_value': 1.0, 'discrete': True, 'likelihood_id': nan, 'name': 'mass', 'parameter_type': 'spectral', 'prior_id': nan, 'scaling': 'log10', 'transform_scale': 31, 'axis': array([  0.1       ,   0.12589254,   0.15848932,   0.19952623,
         0.25118864,   0.31622777,   0.39810717,   0.50118723,
         0.63095734,   0.79432823,   1.        ,   1.25892541,
         1.58489319,   1.99526231,   2.51188643,   3.16227766,
         3.98107171,   5.01187234,   6.30957344,   7.94328235,
        10.        ,  12.58925412,  15.84893192,  19.95262315,
        25.11886432,  31.6227766 ,  39.81071706,  50.11872336,
        63.09573445,  79.43282347, 100.        ]), 'bounds': array([  0.1, 100. ])}


In [12]:
loaded_mass_parameter.axis

array([  0.1       ,   0.12589254,   0.15848932,   0.19952623,
         0.25118864,   0.31622777,   0.39810717,   0.50118723,
         0.63095734,   0.79432823,   1.        ,   1.25892541,
         1.58489319,   1.99526231,   2.51188643,   3.16227766,
         3.98107171,   5.01187234,   6.30957344,   7.94328235,
        10.        ,  12.58925412,  15.84893192,  19.95262315,
        25.11886432,  31.6227766 ,  39.81071706,  50.11872336,
        63.09573445,  79.43282347, 100.        ])

In [13]:
import os

os.system("rm -rf mass_param.h5")

0

If in particular you need to save a `Parameter` class instance with a custom transform function, which you can specify via the `custom_parameter_transform` value of the given dictionary, you will need to use the `save_to_pickle` and `load_from_pickle` methods, or pickle the class instance yourself.

If you can please save to `h5` format to save space, but you do you.

In [19]:
def times2(x):
    return x*2

mass_parameter.transform = times2

mass_parameter.save_to_pickle('pickled_mass.pkl')

In [20]:
loaded_mass_parameter = Parameter.load_from_pcikle('pickled_mass.pkl')

In [22]:
print(loaded_mass_parameter.transform.__name__)
loaded_mass_parameter.transform(1.0)

times2


2.0

# `ParameterSet`

This class is a little more self-explanatory, it contains a set of the `Parameter` classes.